<a href="https://colab.research.google.com/github/DOHYEONAN/git_practice/blob/master/DynamicProgramming_Local_alignment_Protein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protein alignment example from Durbin, et al. (1998)

In [0]:
# Download the scoring matrix 
!wget https://raw.githubusercontent.com/taejoonlab/BME203.2019/master/data/BLOSUM62.txt
!ls

--2019-03-10 16:54:38--  https://raw.githubusercontent.com/taejoonlab/BME203.2019/master/data/BLOSUM62.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2122 (2.1K) [text/plain]
Saving to: ‘BLOSUM62.txt.4’

BLOSUM62.txt.4      100%[===================>]   2.07K  --.-KB/s    in 0s      

2019-03-10 16:54:38 (37.2 MB/s) - ‘BLOSUM62.txt.4’ saved [2122/2122]

BLOSUM62.txt	BLOSUM62.txt.2	BLOSUM62.txt.4
BLOSUM62.txt.1	BLOSUM62.txt.3	sample_data


In [0]:
# Check the matrix
!head -n 20 BLOSUM62.txt
!tail -n 5 BLOSUM62.txt

#  Matrix made by matblas from blosum62.iij
#  * column uses minimum score
#  BLOSUM Clustered Scoring Matrix in 1/2 Bit Units
#  Blocks Database = /data/blocks_5.0/blocks.dat
#  Cluster Percentage: >= 62
#  Entropy =   0.6979, Expected =  -0.5209
   A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *
A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4 
R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4 
N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4 
D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4 
C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4 
Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4 
E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4 
G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4 
H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4 
I -

In [0]:
# Prepare the scoring matrix 'scores'

scores = dict()

f_mat = open('BLOSUM62.txt', 'r')
aa_tokens = []
for line in f_mat:
  if line.startswith('#'):
    continue
  elif line.startswith(' '):
    aa_tokens = line.strip().split()
  else:
    tmp_tokens = line.strip().split()
    for i in range(1, len(tmp_tokens)):
      scores['%s%s' % (aa_tokens[i-1], tmp_tokens[0])] = int(tmp_tokens[i])
f_mat.close()

print(scores['*H'], scores['YY'], scores['WW'])

-4 7 11


In [0]:
# Prepare the input sequences
seq1 = 'HEAGAWGHEE'
seq2 = 'PAWHEAE'

# Prepare the matrix for (1) score and (2) directionality
score_matrix = []
direction_matrix = []

for i in range(0, len(seq2)+1):
  tmp_score_list = []
  tmp_direction_list = []
  for j in range(0, len(seq1)+1):
    tmp_score_list.append(0)
    tmp_direction_list.append('N')
  score_matrix.append(tmp_score_list)
  direction_matrix.append(tmp_direction_list)

# Beautify the matrix output
def print_str_matrix(tmp_mat):
  for tmp_line in tmp_mat:
    print(" ".join([x.ljust(3, ' ') for x in tmp_line]))

def print_matrix(tmp_mat):
  for tmp_line in tmp_mat:
    print(tmp_line)

    
print_matrix(score_matrix)
print_str_matrix(direction_matrix)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  
N   N   N   N   N   N   N   N   N   N   N  


In [0]:
# Fill the matrix

for j in range(0, len(seq2)+1):
  for i in range(0, len(seq1)+1):
    if i == 0 and j == 0:
      score_matrix[i][j] = 0
    elif i == 0:
      # First column
      tmp_aa = '%s%s' % ('*', seq2[j-1])
      score_matrix[j][i] = 0
      direction_matrix[j][i] = ''
    elif j == 0:
      # First row
      tmp_aa = '%s%s' % (seq1[i-1], '*')
      score_matrix[j][i] = 0
      direction_matrix[j][i] = ''
    else:
      # For score
      tmp_aa_L = '%s%s' % (seq1[i-1], '*')
      tmp_score_L = score_matrix[j][i-1] + scores[tmp_aa_L]
      tmp_aa_T = '%s%s' % ('*', seq2[j-1])
      tmp_score_T = score_matrix[j-1][i] + scores[tmp_aa_T]
      tmp_aa_D = '%s%s' % (seq1[i-1], seq2[j-1])
      tmp_score_D = score_matrix[j-1][i-1] + scores[tmp_aa_D]
      
      max_score = max([tmp_score_L, tmp_score_T, tmp_score_D, 0])
      score_matrix[j][i] = max_score
      
      # For direction
      tmp_direction = ''
      if max_score == tmp_score_L:
        tmp_direction += 'L'
      elif max_score == tmp_score_T:
        tmp_direction += 'T'
      if max_score == tmp_score_D:
        tmp_direction += 'D'
      direction_matrix[j][i] = tmp_direction

print("Score Matrix")
print_matrix(score_matrix)

print("Direction Matrix")
print_str_matrix(direction_matrix)

Score Matrix
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2, 0, 15, 11, 7, 3, 0]
[0, 8, 4, 0, 0, 0, 11, 13, 19, 15, 11]
[0, 4, 13, 9, 5, 1, 7, 9, 15, 24, 20]
[0, 0, 9, 17, 13, 9, 5, 7, 11, 20, 23]
[0, 0, 5, 13, 15, 12, 8, 4, 7, 16, 25]
Direction Matrix
N                                          
                                           
            D   LD  D   L   D              
            T   D   T   D   L   L   L      
    D   L   L       D   T   D   D   L   L  
    T   D   L   L   L   T   TD  T   D   LD 
    T   T   D   L   LD  L   D   T   T   D  
    D   TD  T   D   D   L   L   TD  TD  D  


In [0]:
def opt ():
  import random
  
  # First, make two empty lists for each sequences to include information about the optimum alignment. 
  optimum_1 = []
  optimum_2 = []
  
  # Now we start to look for the optimum alignment from the bottom right corner.
  # So, we set the current position to the element at the bottom right corner
  #current_location_1 = len(seq1)
  #current_location_2 = len(seq2)
  
  score = []
  
    
  # Finding max value
  compare_max = []
  
  for one_list in score_matrix:
    compare_max.append(max(one_list))
  max_value = max(compare_max)
  
  #print(max_value)
  
  for j in range(len(seq2)+1):
    for i in range(len(seq1)+1):
      if score_matrix[j][i] == max_value:
        #print(j)
        #print(i)
        current_location_2 = j
        current_location_1 = i
        
  print("Optimum alignment scores " + str(max_value)+ ":" )
  
  while True:
    direction = direction_matrix[current_location_2][current_location_1]    
    if direction == 'TD' or direction =='LD' or direction =='LTD' or direction == 'LT':
      randombox = []
      for ele in range(len(direction)):
        randombox.append(direction[ele])
        
      direction_matrix[current_location_2][current_location_1] = random.choice(randombox)
  
    # Now we define what the first element is in optimum alignment.
    # It depends on what direction the first element in matrix has.  
    if direction_matrix[current_location_2][current_location_1] == 'D':
      optimum_1.append(seq1[current_location_1 - 1])
      optimum_2.append(seq2[current_location_2 - 1])
      
      score.append(score_matrix[current_location_2][current_location_1] - score_matrix[current_location_2 - 1][current_location_1 - 1])
      
      current_location_2 -= 1
      current_location_1 -= 1
      #Local alignment
      if score_matrix[current_location_2][current_location_1] == 0:
        break
  
    if direction_matrix[current_location_2][current_location_1] == 'L':
      optimum_1.append(seq1[current_location_1 - 1])
      optimum_2.append('-')
      
      score.append(score_matrix[current_location_2][current_location_1] - score_matrix[current_location_2][current_location_1 - 1])
      
      current_location_2 -= 0
      current_location_1 -= 1
      #Local alignment
      if score_matrix[current_location_2][current_location_1] == 0:
        break
  
    if direction_matrix[current_location_2][current_location_1] == 'T':
      optimum_1.append('-')
      optimum_2.append(seq2[current_location_2 - 1])
      
      score.append(score_matrix[current_location_2][current_location_1] - score_matrix[current_location_2 - 1][current_location_1])
      
      current_location_2 -= 1
      current_location_1 -= 0
      #Local alignment
      if score_matrix[current_location_2][current_location_1] == 0:
        break
     
  reverse_optimum_1 = optimum_1[::-1]
  reverse_optimum_2 = optimum_2[::-1]
  reverse_score = score[::-1]
  

    #print(optimum_2, end = " ")
    #print(optimum_1)
  
    #a='ABCD'
    #print(a[::-1])
    #prunt(a[0:2])
    #print(a[-2:])
  
    #print(reverse_optimum_2)
    #print(reverse_optimum_1)
  
  return reverse_optimum_1, reverse_optimum_2, reverse_score


  
reverse_optimum_1,reverse_optimum_2, score = opt()


for ele in reverse_optimum_2:
  print(ele, end = "     ")
print('')
  
for ele in reverse_optimum_1:
  print(ele, end = "     ")
print('')

for ele in score:
  print(ele, end = "    ")
print('')



Optimum alignment scores 25:
A     W     -     H     E     A     E     
A     W     G     H     E     -     E     
4    11    -4    8    5    -4    5    
